<div align="right"><i>Peter Norvig<br>2012; updated 18 August 2019</i></div>

# Twelve Balls and a Balance Scale

> *You are given twelve identical-looking balls and a two-sided scale. One of the balls is of a different weight, although you don't know whether it's lighter or heavier. How can you use just three weighings of the scale to determine not only what the different ball is, but also whether it's lighter or heavier?*

This is a traditional brain-teaser puzzle, meant to be solved with paper and pencil. 
But I want to not just solve this specific puzzle, but show how to write a program that can solve related puzzles where you can vary (a) the number of balls, (b) the number of weighings allowed, and (c) whether the odd ball might be heavier, lighter, or either. (I originally solved this in 2012, but am republishing it here in revised form because the problem was mentioned in the [538 Riddler](https://fivethirtyeight.com/features/which-billiard-ball-is-rigged/) for 16 August 2019.)

# Design

Here are the concepts I'm dealing with:

- **balls**: In the general case I have N balls; for example I'll represent N = 3 balls with `[1, 2, 3]`.
- **oddballs**: Exactly one of the balls is **odd** in its weight; if ball N is heavier, I'll represent that as +N; if it is lighter, as -N. With N = 3, I will represent the collection of possible oddballs as `[+1, -1, +2, -2, +3]`.
- **puzzle**: A specific puzzle declares the number of balls, the maximum number of weighings allowed, and the oddballs. 
- **weighing**: I can weigh a collection of balls on the left versus a collection on the right, and the result (given that we know the oddball) will be that the left side is greater than, equal to, or less than the right in weight.
I'll denote that with the call `weigh(L, R, oddball)`, which returns a string, `'gt'`, `'eq'`, or `'lt'`.
- **weight**: I'll arbitrarily say that a normal ball weighs 100, a lighter ball 99, and a heavier ball 101.
- **solution**: A particular puzzle states the number of balls, the number of weighings allowed, and whether the odd ball can be lighter, heavier, or either. For example, the original puzzle is solved with a call to `solve(12, 3, {+1, -1})`. The solution is a **strategy tree**.
- **strategy tree**: a tree where each node is either a leaf node consisting of an *oddball* integer, or is an interior node with 5 components: the balls to be placed on the left and right side of the scale, and a subtree for each of the three possible outcomes: `'gt'`, `'eq'`, or `'lt'`. The constructor `Tree(L, R, gt, eq, lt)` creates a tree, where `L` and `R` are collections of balls, and `gt`, `eq`, and `lt` are trees.
- **following a path in a tree**: I'll use `follow(tree, oddball)` to say "follow the path through the tree, at each weighing assuming the given oddball, and return the leaf node reached&mdash;the oddball that the tree predicts.
- **valid tree**: a tree is a valid solution if no branch uses more than the allowable number of weighings, and if, for every possible oddball, following the path through the tree gives the correct oddball as the answer. We'll delay the discussion of how to find a valid tree until later.


# Implementation

Let's start implementing:

In [1]:
from collections import namedtuple
import random

#### Types

class Puzzle:
    "Represent a specific ball-weighing puzzle."
    def __init__(self, N=12, weighings=3, oddities={+1, -1}):
        self.N = N
        self.weighings = weighings
        self.balls = list(range(1, N + 1))
        self.oddballs = [b * o for b in self.balls for o in oddities]  
        
Tree = namedtuple('Tree', 'L, R, gt, eq, lt')

Oddball = int

#### Functions
    
def weigh(L, R, oddball) -> str:
    "Weigh balls L against balls R, given the oddball; return 'gt', 'eq', or 'lt'."
    diff = sum(weight(b, oddball) for b in L) - sum(weight(b, oddball) for b in R)
    return ('gt' if diff > 0 else
            'lt' if diff < 0 else
            'eq')

def weight(ball, oddball) -> int: 
    return 101 if +ball == oddball else 99 if -ball == oddball else 100
    
def solve(puzzle) -> Tree:
    "Return a valid tree; one that solves the puzzle."
    tree = find_tree(puzzle, puzzle.oddballs, puzzle.weighings)
    assert valid(tree, puzzle)
    return tree
    
def follow(tree, oddball) -> Oddball:
    "Follow a path through the tree and return the oddball that the tree leads us to."
    if isinstance(tree, Oddball):
        return tree
    else:
        result = weigh(tree.L, tree.R, oddball)
        return follow(getattr(tree, result), oddball)
    
def valid(tree, puzzle) -> bool:
    "Does the strategy tree solve the puzzle correctly for all possible oddballs?"
    return (depth(tree) <= puzzle.weighings and 
            all(follow(tree, oddball) == oddball for oddball in puzzle.oddballs))

def depth(tree) -> int:
    "Maximum depth of a strategy tree."
    return (0 if isinstance(tree, Oddball) else 
            1 + max(depth(tree.gt), depth(tree.eq), depth(tree.lt)))

Let's try out these functions:

In [2]:
p12 = Puzzle(12) # The original puzzle with 12 balls

print(p12.weighings)
print(p12.balls)
print(p12.oddballs)

3
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
[1, -1, 2, -2, 3, -3, 4, -4, 5, -5, 6, -6, 7, -7, 8, -8, 9, -9, 10, -10, 11, -11, 12, -12]


In [3]:
# If we weigh balls 1, 2 against 3, 4, and the oddball is that 5 is lighter, the result should be 'eq'
weigh([1, 2], [3, 4], -5)

'eq'

# Strategy for Finding a Valid Tree

Now for the tricky part. We want to find a valid tree to solve a puzzle. The key idea is that a **weighing** gives us information by making a **partition** of the possible **oddballs**; subsequent subtrees can handle each of the partitions.


In [4]:
def partition(L, R, oddballs) -> dict:
    "Give all the possible outcomes of weighing L versus R."
    part = dict(gt=[], eq=[], lt=[])
    for odd in oddballs:
        part[weigh(L, R, odd)].append(odd)
    return part

For example, with 12 balls, if we weigh balls 1 and 2 on the left versus 11 and 12 on the right, then there are four ways the left side can be greater than the right: either 1 or 2 is heavier or 11 or 12 is lighter. The opposite would lead to the left side being less than the right. And if any of balls 3 through 12 is either heavier or lighter, the result would be that the weighing is equal.

In [5]:
partition([1, 2], [11, 12], p12.oddballs)

{'gt': [1, 2, -11, -12],
 'eq': [3, -3, 4, -4, 5, -5, 6, -6, 7, -7, 8, -8, 9, -9, 10, -10],
 'lt': [-1, -2, 11, 12]}

Given that the puzzle is to solve the problem in 3 weighings, we call this a **bad partition**, because the `'eq'` entry has 16 possibilities, which is too many to solve in the remaining 2 weighings. Each weighing can at best partition the possibilities into three equal groups. So with two remaining weighings, we can only handle up to 3 &times; 3 = 9 possible oddballs, not 16.

The following is a **good partition** because each of the entries has 8 possibilities, and 8 is less than 9. (Note: being a good partition does not guarantee that the problem is solvable from there; but being a bad partition guarantees that it is not.)

In [6]:
partition([1, 2, 3, 4], [9, 10, 11, 12], p12.oddballs)

{'gt': [1, 2, 3, 4, -9, -10, -11, -12],
 'eq': [5, -5, 6, -6, 7, -7, 8, -8],
 'lt': [-1, -2, -3, -4, 9, 10, 11, 12]}

So now we have a viable approach to implementing `find_tree`, which is the core of `solve`:

   - We call `find_tree(puzzle, oddballs, weighings)`. At the top level, the oddballs and number of weighings come from the puzzle. At recursive levels, we will reduce the number of oddball possibilities according to the partition, and the number of remaining weighings by 1 each time.
   - At each step we will randomly select two groups of balls, `L` and `R`, to be weighed.
   - We will then see what partition `L` and `R` gives us, and whether the partition is good or bad.
   - We will adopt a **greedy** approach where we accept the first good partition.
      (If we don't find a good partition after 10,000 tries, we give up.)
   - Once we have a good partition, we recursively find a tree for each of the branches of the partition.

In [7]:
def find_tree(puzzle, oddballs, weighings):
    "Find a strategy tree that covers all the oddballs in the given number of weighings."
    if len(oddballs) == 1:
        return oddballs[0] # One oddball possibility left; we're done
    elif len(oddballs) == 0 or weighings == 0:
        return 0           # No valid strategy or an impossible situation
    else:
        L, R, part = good_partition(puzzle, oddballs, weighings - 1)
        return Tree(L, R, **{r: find_tree(puzzle, part[r], weighings - 1) for r in part})
    
def good_partition(puzzle, oddballs, weighings):
    "Randomly pick L, R balls such that no partition entry has more than 3**weighings oddballs."
    for _ in range(10000): 
        L, R = random_LR(puzzle, oddballs)
        part = partition(L, R, oddballs)
        good = all(len(entry) <= 3 ** weighings for entry in part.values())
        if good:
            return L, R, part
    raise ValueError('good_partition not found')
    
def random_LR(puzzle, oddballs):
    "Random choice of balls for L and R side."
    # Pick a random number of balls, B, then pick B balls for each side.
    B = random.choice(range(1, (len(puzzle.balls) - 1) // 3 + 2))
    random.shuffle(puzzle.balls) 
    return sorted(puzzle.balls[:B]), sorted(puzzle.balls[-B:])

Here we see that `good_partition` does its job:

In [8]:
good_partition(p12, p12.oddballs, 2)

([5, 10, 11, 12],
 [1, 2, 4, 8],
 {'gt': [-1, -2, -4, 5, -8, 10, 11, 12],
  'eq': [3, -3, 6, -6, 7, -7, 9, -9],
  'lt': [1, 2, 4, -5, 8, -10, -11, -12]})

But it uses `random`, so it won't get the same result every time:

In [9]:
good_partition(p12, p12.oddballs, 2)

([3, 4, 9, 12],
 [1, 5, 8, 11],
 {'gt': [-1, 3, 4, -5, -8, 9, -11, 12],
  'eq': [2, -2, 6, -6, 7, -7, 10, -10],
  'lt': [1, -3, -4, 5, 8, -9, 11, -12]})

Now we're ready to solve puzzles!

In [10]:
solve(p12)

Tree(L=[2, 3, 6, 8], R=[1, 4, 9, 10], gt=Tree(L=[6, 8, 10], R=[2, 3, 11], gt=Tree(L=[3, 6, 10], R=[1, 2, 4], gt=6, eq=8, lt=0), eq=Tree(L=[9, 10], R=[4, 5], gt=-4, eq=-1, lt=-9), lt=Tree(L=[3, 7], R=[2, 5], gt=3, eq=-10, lt=2)), eq=Tree(L=[1, 5, 8, 12], R=[2, 4, 7, 9], gt=Tree(L=[5, 8, 10], R=[1, 3, 12], gt=5, eq=-7, lt=12), eq=Tree(L=[4, 7, 12], R=[2, 10, 11], gt=-11, eq=0, lt=11), lt=Tree(L=[2, 5, 7], R=[1, 4, 9], gt=7, eq=-12, lt=-5)), lt=Tree(L=[3, 6, 10, 11], R=[2, 5, 7, 8], gt=Tree(L=[6, 8, 10], R=[1, 11, 12], gt=10, eq=-2, lt=-8), eq=Tree(L=[1, 5, 7, 11], R=[2, 3, 4, 12], gt=1, eq=9, lt=4), lt=Tree(L=[3, 8, 10], R=[2, 7, 12], gt=0, eq=-6, lt=-3)))

OK, that's hard to read&mdash;my bad. Let's look at an easier puzzle (3 balls in 2 weighings):

In [11]:
solve(Puzzle(3, 2))

Tree(L=[3], R=[1], gt=Tree(L=[2], R=[3], gt=0, eq=-1, lt=3), eq=Tree(L=[3], R=[2], gt=-2, eq=0, lt=2), lt=Tree(L=[2], R=[3], gt=-3, eq=1, lt=0))

You should be able to follow through that tree by hand, if you're careful. But let's make the trees easier to read by formatting them:

In [12]:
def show(tree):
    "Print an indented tree."
    print(pp(tree))
    
def pp(tree, i=0, suffix=''):
    "Pretty, indented string of a strategy tree."
    if isinstance(tree, Tree):
        indent = '' if i == 0 else ('\n' + " " * 5 * i)
        return f'{indent}Tree({tree.L}, {tree.R}, {pp(tree.gt, i+1)}, {pp(tree.eq, i+1)}, {pp(tree.lt, i+1)})'
    else:
        return f'{tree:+d}'

In [13]:
show(solve(Puzzle(3)))

Tree([2], [1], 
     Tree([2], [1], 
          Tree([3], [1], -1, +2, +0), +0, +0), 
     Tree([1], [3], -3, +0, +3), 
     Tree([3], [1], +0, -2, +1))


In [14]:
# Back to the original puzzle
show(solve(p12))

Tree([3, 6, 7, 9], [2, 5, 10, 12], 
     Tree([9, 10, 12], [2, 7, 11], 
          Tree([2], [1], +0, +9, -2), 
          Tree([4, 6, 9], [2, 3, 10], +6, -5, +3), 
          Tree([3, 10], [9, 12], -12, +7, -10)), 
     Tree([4, 5, 6, 11], [2, 8, 10, 12], 
          Tree([4, 8], [5, 10], +4, +11, -8), 
          Tree([1, 6, 10, 11], [3, 4, 7, 12], +1, +0, -1), 
          Tree([4, 10], [5, 11], -11, +8, -4)), 
     Tree([3, 4, 5, 7], [1, 8, 9, 12], 
          Tree([6], [5], +0, -9, +5), 
          Tree([1, 2, 6], [5, 8, 11], +2, +10, -6), 
          Tree([4, 8, 9], [1, 3, 12], -3, -7, +12)))


In [15]:
# You get different solutions on different runs.
show(solve(p12))

Tree([6, 7, 11, 12], [2, 5, 8, 9], 
     Tree([2, 11, 12], [3, 6, 9], 
          Tree([1, 5, 12], [2, 3, 11], +12, -9, +11), 
          Tree([1, 5, 7, 9], [2, 3, 4, 11], +7, -8, -5), 
          Tree([1, 7, 9, 11], [3, 6, 8, 10], +0, -2, +6)), 
     Tree([2, 3, 6, 11], [1, 5, 8, 10], 
          Tree([1], [10], -10, +3, -1), 
          Tree([1, 4, 6, 8], [2, 10, 11, 12], +4, +0, -4), 
          Tree([5, 7, 12], [1, 2, 3], -3, +10, +1)), 
     Tree([3, 4, 5, 10], [2, 6, 8, 9], 
          Tree([3, 7, 11, 12], [1, 4, 5, 9], +0, -6, +5), 
          Tree([2, 3, 9, 11], [5, 8, 10, 12], -12, -7, -11), 
          Tree([6, 7, 9, 10], [3, 5, 8, 12], +9, +2, +8)))


We can solve larger puzzles with 4 weighings, but before showing that, let's do two things.
- Condense each line a bit by replacing `', '` with just `','`. This makes more fit on one line.
- At the top level, there's no sense randomly shuffling the balls; the only choice that matters is how many balls, `B`, to put on each side.

In [16]:
def show(tree):
    "Print a condensed representation tree."
    print(pp(tree).replace(', ', ','))
    
def random_LR(puzzle, oddballs):
    "Random choice of balls for L and R side."
    # Pick a random number of balls, B, then pick B balls for each side.
    B = random.choice(range(1, (len(puzzle.balls) - 1) // 3 + 2))
    if oddballs == puzzle.oddballs:
        puzzle.balls.sort()
    else:
        random.shuffle(puzzle.balls) 
    return sorted(puzzle.balls[:B]), sorted(puzzle.balls[-B:])

In [17]:
show(solve(Puzzle(39, 4)))

Tree([1,2,3,4,5,6,7,8,9,10,11,12,13],[27,28,29,30,31,32,33,34,35,36,37,38,39],
     Tree([2,3,6,8,12,18,19,24,27,31,34,36,39],[7,11,13,15,16,20,22,25,26,28,30,33,37],
          Tree([3,6,8,9,14,25,26,30,31,36,37,39],[1,2,4,5,10,13,17,18,19,22,23,29],
               Tree([3,13,21,23,27,30,31,32,38],[1,2,4,6,9,14,25,37,39],+3,+8,+6),
               Tree([1,7,9,11,16,23,26,28,30,39],[6,10,13,14,15,19,21,32,33,38],-33,+12,-28),
               Tree([2,10,11,12,13,22,23,26,30,31,34],[3,7,14,18,19,21,24,27,29,38,39],+2,-37,-30)),
          Tree([2,15,18,21,22,23,26,27,28,31,33,34,39],[1,4,10,12,16,17,19,25,29,30,35,37,38],
               Tree([10,14,17,18,21,22,27,30,32,34,35],[1,5,7,9,12,13,19,24,28,33,38],-38,-29,-35),
               Tree([2,7,9,10,18,23,26,27,28,31,36,38],[3,5,6,11,14,16,19,21,24,25,35,39],+9,-32,+5),
               Tree([1,16,17,21,25,36,38,39],[7,10,12,14,20,28,30,33],+1,+4,+10)),
          Tree([2,8,15,16,23,25,31,32],[11,13,18,19,29,30,34,36],
               Tree([6,19

We can get up to 26 balls in 3 weighings if we know that the only possibility is that one is lighter (no ball can be heavier):

In [18]:
show(solve(Puzzle(26, 3, {-1})))

Tree([1,2,3,4,5,6,7,8,9],[18,19,20,21,22,23,24,25,26],
     Tree([6,7,10,11,13,20,22,23],[1,3,4,5,9,18,21,26],
          Tree([1,6,11,13,19,21,22,23],[2,5,7,8,14,17,20,26],-26,-18,-21),
          Tree([1,7,11,15,18,20,21,23,24],[2,3,4,10,13,14,19,22,26],-19,-25,-24),
          Tree([7,8,12,13,18,20,25,26],[1,3,5,6,14,16,21,22],-22,-23,-20)),
     Tree([6,8,13,15,18,20,21,23],[1,4,9,14,16,17,19,22],
          Tree([8,11,16,21],[5,12,17,20],-17,-14,-16),
          Tree([6,10,26],[2,11,16],-11,-12,-10),
          Tree([1,26],[13,18],-13,-15,+0)),
     Tree([5,8,9,10,15,23,24,26],[1,6,7,12,13,18,20,22],
          Tree([1,5,9,16,17,20,24,25],[2,3,6,8,12,13,14,19],-6,-7,-1),
          Tree([1,2,9,10,14,18,22,23,26],[3,6,7,11,12,13,20,21,25],-3,-4,-2),
          Tree([8,14,16,23],[4,5,6,19],-5,-9,-8)))


What other puzzles can you solve?